# JsonOutputParser

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [4]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [5]:
# 원하는 데이터 구조를 정의
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [6]:
# 질의 작성
question = "지구 온난화의 심각성에 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\nQuestion: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

chain.invoke(question)

{'description': '지구 온난화는 기후 변화의 주요 원인으로, 지구의 평균 기온 상승을 초래하여 생태계와 인류에 심각한 영향을 미치고 있습니다.',
 'hashtags': '#지구온난화 #기후변화'}

# Pydantic 없이 사용하기

In [7]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 'description'에, 관련 키워드는 'hashtags'에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\nQuestion: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | model | parser

response = chain.invoke({"question": question})

response

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 온실가스의 증가로 인해 발생합니다. 이로 인해 기후 변화, 해수면 상승, 생태계 파괴 등의 문제가 발생할 수 있습니다.',
 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#지구환경']}